In [22]:
# load packages

import time

from IPython import get_ipython
from IPython.display import clear_output
import os
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse import csc_matrix
from scipy import signal as sg
import scipy
import pickle

from tifffile.tifffile import imwrite,imread
from tqdm.auto import tqdm,trange

from copy import deepcopy
import h5py

import caiman as cm
from caiman.source_extraction.cnmf import cnmf,params
from caiman.paths import caiman_datadir
from caiman.utils.visualization import get_contours

try:
    if __IPYTHON__:
        get_ipython().run_line_magic('load_ext', 'autoreload')
        get_ipython().run_line_magic('autoreload', '2')
except NameError:
    pass

def load_pickle(file_path):
    """
    Load a dictionary from a pickle file.

    Args:
    - file_path (str): Path to the pickle file.

    Returns:
    - dict: Loaded dictionary.
    """
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
    return data

def flatten_extend(matrix):
     flat_list = []
     for row in matrix:
         flat_list.extend(row)
     return flat_list

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
# EDITED FURTHER 7/20/24 with pythonic order to properly coordinate matches!
# EDITED 5/5/24 with doraw option, also increased nrg_thr to 0.95 (not updated on Megatron/Beastmode)
data_dir_l = [#r'\\nasquatch\data\2p\peter\in_vivo\220903_PK63PL34\mouse\preprocessing\220903_PK63PL34_mouse_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\in_vivo\220903_PK63PL34\wells101\preprocessing\220903_PK63PL34_wells101_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\in_vivo\230615_DG69PL32\mouse\preprocessing\230615_DG69PL32_mouse_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\in_vivo\230615_DG69PL32\wells101\preprocessing\230615_DG69PL32_wells101_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\in_vivo\230720_PK97PL48\mouse\preprocessing\230720_PK97PL48_mouse_Yael_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\in_vivo\230720_PK97PL48\wells101\preprocessing\230720_PK97PL48_wells101_Yael_done_qc_doneqc',
             r'\\nasquatch\data\2p\peter\in_vivo\230802_DG48PL58\mouse\preprocessing\230802_DG48PL58_mouse_Done_qc_doneqc',
             r'\\nasquatch\data\2p\peter\in_vivo\230802_DG48PL58\wells101\preprocessing\230802_DG48PL58_wells101_Done_qc_doneqc']
             #r'\\nasquatch\data\2p\peter\in_vivo\231018_DG52PL66\mouse\preprocessing\231018_DG52PL66_mouse_Tiara_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\in_vivo\231018_DG52PL66\wells101\preprocessing\231018_DG52PL66_wells101_Tiara_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\in_vivo\231030_DG60PL67\mouse\preprocessing\231030_DG60PL67_mouse_Yael_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\in_vivo\231030_DG60PL67\wells\preprocessing\231030_DG60PL67_wells_maddy_done_qc_done']
mmap_dir_l = [#r'D:\pkalugin\memmap_cache\PK63PL34_220903_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref55_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PK63PL34_220903_101_full_wch0_aniso4_scale4AFS2PL4_toref115_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\DG69PL32_230615_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref71_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\DG69PL32_230615_101_full_wch0_aniso4_scale4AFS2PL4_toref54_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PK97PL48_230720_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref70_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PK97PL48_230720_101_full_wch0_aniso4_scale4AFS2PL4_toref50_wxy_nobg',
             r'D:\pkalugin\memmap_cache\DG48PL58_230802_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref71_wxy_nobg',
             r'D:\pkalugin\memmap_cache\DG48PL58_230802_101_full_wch0_aniso4_scale4AFS2PL4_toref57_wxy_nobg']
             #r'D:\pkalugin\memmap_cache\DG52PL66_231018_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref71_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\DG52PL66_231018_101_full_wch0_aniso4_scale4AFS2PL4_toref34_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\DG60PL67_231030_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref49_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\DG60PL67_231030_101_full_wch1_aniso4_scale4AFS2PL4_toref34_wxy_nobg']
mov_means_l = [52,52]
pythonic_l = [0,0]

doraw = True
n_cores = 30
nrg_thr = 0.95

# Parameters (unlikely to change)
n_range_lim = 10 # size of n_range below which SNR considered unreliable
Athresh = 0.05 # overlap threshold - automatically split anything below it
cr_thresh = 0.9 # component-raw correlation threshold below which component deemed suspicious quality
pb_thresh = 0.95 # component-best parent correlation threshold above which component deemed likely merge

In [20]:
# EDITED FURTHER 7/20/24 with pythonic order to properly coordinate matches!
# EDITED 5/5/24 with doraw option, also increased nrg_thr to 0.95 (not updated on Megatron/Beastmode)
data_dir_l = [#r'\\nasquatch\data\2p\peter\dipping\20230927_PL34\wells001\preprocessing\20230927_PL34_wells001_david_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230927_PL34\wells001\preprocessing\20230927_PL34_wells001_david_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230927_PL34\wells001\preprocessing\20230927_PL34_wells001_david_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230927_PL34\wells101\preprocessing\20230927_PL34_wells101_Done_qc_doneqc',
             r'\\nasquatch\data\2p\peter\dipping\20230927_PL34\wells201\preprocessing\20230927_PL34_wells201_david_done_qc_doneqc',
             r'\\nasquatch\data\2p\peter\dipping\20230927_PL34\wells301\preprocessing\20230927_PL34_wells301_Done_qc_doneqc']
             #r'\\nasquatch\data\2p\peter\in_vivo\220622_PK44PL17\mouse\preprocessing\220622_PK44PL17_mouse_Tiara_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\in_vivo\220622_PK44PL17\wells101\preprocessing\220622_PK44PL17_wells101_Tiara_done_qc_doneqc']
mmap_dir_l = [#r'D:\pkalugin\memmap_cache\PL34_230927_001_full_wch0_aniso4_scale4AFS2PL4_toref20_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL34_230927_001_full_wch0_aniso4_scale4AFS2PL4_toref20_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL34_230927_001_full_wch0_aniso4_scale4AFS2PL4_toref20_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL34_230927_101_full_wch0_aniso4_scale4AFS2PL4_toref35_wxy_nobg',
             r'D:\pkalugin\memmap_cache\PL34_230927_201_full_wch0_aniso4_scale4AFS2PL4_toref37_wxy_nobg',
             r'D:\pkalugin\memmap_cache\PL34_230927_301_full_wch0_aniso4_scale4AFS2PL4_toref30_wxy_nobg']
             #r'D:\pkalugin\memmap_cache\PK44PL17_220622_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref76_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PK44PL17_220622_101_full_wch0_aniso4_scale4AFS2PL4_toref115_wxy_nobg']
mov_means_l = [31,
              31]
pythonic_l = [#0,1,2,0,
              1,2]
             #0,0]

doraw = True
n_cores = 30
nrg_thr = 0.95

# Parameters (unlikely to change)
n_range_lim = 10 # size of n_range below which SNR considered unreliable
Athresh = 0.05 # overlap threshold - automatically split anything below it
cr_thresh = 0.9 # component-raw correlation threshold below which component deemed suspicious quality
pb_thresh = 0.95 # component-best parent correlation threshold above which component deemed likely merge

In [17]:
# EDITED FURTHER 7/20/24 with pythonic order to properly coordinate matches!
# EDITED 5/5/24 with doraw option, also increased nrg_thr to 0.95 (not updated on Megatron/Beastmode)
data_dir_l = [#r'\\nasquatch\data\2p\peter\dipping\20230926_PL56\wells001\preprocessing\20230926_PL56_wells001_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230926_PL56\wells001\preprocessing\20230926_PL56_wells001_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230926_PL56\wells001\preprocessing\20230926_PL56_wells001_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230926_PL56\wells001\preprocessing\20230926_PL56_wells001_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230926_PL56\wells001\preprocessing\20230926_PL56_wells001_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230926_PL56\wells101\preprocessing\20230926_PL56_wells101_Done_qc_doneqc',
             r'\\nasquatch\data\2p\peter\dipping\20230926_PL56\wells201\preprocessing\20230926_PL56_wells201_jordan_done_qc_doneqc',
             r'\\nasquatch\data\2p\peter\dipping\20230926_PL56\wells301\preprocessing\20230926_PL56_wells301_jordan_done_qc_doneqc']
             #r'\\nasquatch\data\2p\peter\dipping\20230926_PL56\wells401\preprocessing\20230926_PL56_wells401_david_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230926_PL56\wells501\preprocessing\20230926_PL56_wells501_david_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\in_vivo\220427_PK27PL20\mouse\preprocessing\220427_PK27PL20_mouse_Tiara_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\in_vivo\220427_PK27PL20\wells101\preprocessing\220427_PK27PL20_wells101_Tiara_done_qc_doneqc']
mmap_dir_l = [#r'D:\pkalugin\memmap_cache\PL56_230926_001_full_wch0_aniso4_scale4AFS2PL4_toref25_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL56_230926_001_full_wch0_aniso4_scale4AFS2PL4_toref25_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL56_230926_001_full_wch0_aniso4_scale4AFS2PL4_toref25_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL56_230926_001_full_wch0_aniso4_scale4AFS2PL4_toref25_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL56_230926_001_full_wch0_aniso4_scale4AFS2PL4_toref25_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL56_230926_101_full_wch0_aniso4_scale4AFS2PL4_toref25_wxy_nobg',
             r'D:\pkalugin\memmap_cache\PL56_230926_201_full_wch0_aniso4_scale4AFS2PL4_toref32_wxy_nobg',
             r'D:\pkalugin\memmap_cache\PL56_230926_301_full_wch0_aniso4_scale4AFS2PL4_toref35_wxy_nobg']
             #r'D:\pkalugin\memmap_cache\PL56_230926_401_full_wch0_aniso4_scale4AFS2PL4_toref30_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL56_230926_501_full_wch0_aniso4_scale4AFS2PL4_toref31_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PK27PL20_220427_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref77_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PK27PL20_220427_101_full_wch0_aniso4_scale4AFS2PL4_toref80_wxy_nobg']
mov_means_l = [31,
              31]
pythonic_l = [#0,1,2,3,4,0,
              1,2]#,3,4,
             #0,0]

doraw = True
n_cores = 30
nrg_thr = 0.95

# Parameters (unlikely to change)
n_range_lim = 10 # size of n_range below which SNR considered unreliable
Athresh = 0.05 # overlap threshold - automatically split anything below it
cr_thresh = 0.9 # component-raw correlation threshold below which component deemed suspicious quality
pb_thresh = 0.95 # component-best parent correlation threshold above which component deemed likely merge

In [14]:
# EDITED FURTHER 7/20/24 with pythonic order to properly coordinate matches!
# EDITED 5/5/24 with doraw option, also increased nrg_thr to 0.95 (not updated on Megatron/Beastmode)
data_dir_l = [#r'\\nasquatch\data\2p\peter\dipping\20230922_PL28\wells001\preprocessing\20230922_PL28_wells001_jordan_done_qc_done',
             #r'\\nasquatch\data\2p\peter\dipping\20230922_PL28\wells001\preprocessing\20230922_PL28_wells001_jordan_done_qc_done',
             #r'\\nasquatch\data\2p\peter\dipping\20230922_PL28\wells001\preprocessing\20230922_PL28_wells001_jordan_done_qc_done',
             #r'\\nasquatch\data\2p\peter\dipping\20230922_PL28\wells001\preprocessing\20230922_PL28_wells001_jordan_done_qc_done',
             #r'\\nasquatch\data\2p\peter\dipping\20230922_PL28\wells001\preprocessing\20230922_PL28_wells001_jordan_done_qc_done',
             #r'\\nasquatch\data\2p\peter\dipping\20230922_PL28\wells101\preprocessing\20230922_PL28_wells101_david_done_qc_doneqc',
             r'\\nasquatch\data\2p\peter\dipping\20230922_PL28\wells301\preprocessing\20230922_PL28_wells301_david_done_qc_doneqc',
             r'\\nasquatch\data\2p\peter\dipping\20230922_PL28\wells501\preprocessing\20230922_PL28_wells501_david_done_qc_doneqc',
             r'\\nasquatch\data\2p\peter\dipping\20230922_PL28\wells601\preprocessing\20230922_PL28_wells601_david_done_qc_doneqc']
             #r'\\nasquatch\data\2p\peter\dipping\20230922_PL28\wells701\preprocessing\20230922_PL28_wells701_david_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\in_vivo\220216_PK25PL8\mouse\preprocessing\220216_PK25PL8_mouse_david_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\in_vivo\220216_PK25PL8\wells101\preprocessing\220216_PK25PL8_wells101_david_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\in_vivo\220311_PK24PL9\mouse\preprocessing\220311_PK24PL9_mouse_Tiara_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\in_vivo\220311_PK24PL9\wells101\preprocessing\220311_PK24PL9_wells101_Tiara_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\in_vivo\220406_PK24PL10\mouse\preprocessing\220406_PK24PL10_mouse_jordan_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\in_vivo\220406_PK24PL10\wells101\preprocessing\220406_PK24PL10_wells101_jordan_done_qc_doneqc']
mmap_dir_l = [#r'D:\pkalugin\memmap_cache\PL28_230922_001_full_wch1_aniso4_scale4AFS2PL4_toref35_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL28_230922_001_full_wch1_aniso4_scale4AFS2PL4_toref35_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL28_230922_001_full_wch1_aniso4_scale4AFS2PL4_toref35_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL28_230922_001_full_wch1_aniso4_scale4AFS2PL4_toref35_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL28_230922_001_full_wch1_aniso4_scale4AFS2PL4_toref35_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL28_230922_101_full_wch0_aniso4_scale4AFS2PL4_toref15_wxy_nobg',
             r'D:\pkalugin\memmap_cache\PL28_230922_301_full_wch0_aniso4_scale4AFS2PL4_toref35_wxy_nobg',
             r'D:\pkalugin\memmap_cache\PL28_230922_501_full_wch0_aniso4_scale4AFS2PL4_toref15_wxy_nobg',
             r'D:\pkalugin\memmap_cache\PL28_230922_601_full_wch0_aniso4_scale4AFS2PL4_toref15_wxy_nobg']
             #r'D:\pkalugin\memmap_cache\PL28_230922_701_full_wch0_aniso4_scale4AFS2PL4_toref40_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PK25PL8_220216_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref62_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PK25PL8_220216_101_full_wch0_aniso4_scale4AFS2PL4_toref45_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PK24PL9_220311_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref62_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PK24PL9_220311_101_full_wch0_aniso4_scale4AFS2PL4_toref100_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PK24PL10_220406_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref40_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PK24PL10_220406_101_full_wch0_aniso4_scale4AFS2PL4_toref57_wxy_nobg']
mov_means_l = [#31,
              #31,
              #31,
              #31,
              #31,
              #31,
              31,
              31,
              31]
              #31,
              #52,
              #52,
              #52,
              #52,
              #52,
              #52]
pythonic_l = [#0,2,4,5,6,0,
              2,4,5]#,6,
             #0,0,
             #0,0,
             #0,0]

doraw = True
n_cores = 30
nrg_thr = 0.95

# Parameters (unlikely to change)
n_range_lim = 10 # size of n_range below which SNR considered unreliable
Athresh = 0.05 # overlap threshold - automatically split anything below it
cr_thresh = 0.9 # component-raw correlation threshold below which component deemed suspicious quality
pb_thresh = 0.95 # component-best parent correlation threshold above which component deemed likely merge

In [11]:
# EDITED FURTHER 7/20/24 with pythonic order to properly coordinate matches!
# EDITED 5/5/24 with doraw option, also increased nrg_thr to 0.95 (not updated on Megatron/Beastmode)
data_dir_l = [#r'\\nasquatch\data\2p\peter\dipping\20230927_PL58\wells001\preprocessing\20230927_PL58_wells001_david_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230927_PL58\wells001\preprocessing\20230927_PL58_wells001_david_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230927_PL58\wells001\preprocessing\20230927_PL58_wells001_david_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230927_PL58\wells101\preprocessing\20230927_PL58_wells101_david_done_qc_doneqc',
             r'\\nasquatch\data\2p\peter\dipping\20230927_PL58\wells201\preprocessing\20230927_PL58_wells201_david_done_qc_doneqc',
             r'\\nasquatch\data\2p\peter\dipping\20230927_PL58\wells301\preprocessing\20230927_PL58_wells301_david_done_qc_doneqc']
mmap_dir_l = [#r'D:\pkalugin\memmap_cache\PL58_230927_001_full_wch0_aniso4_scale4AFS2PL4_toref25_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL58_230927_001_full_wch0_aniso4_scale4AFS2PL4_toref25_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL58_230927_001_full_wch0_aniso4_scale4AFS2PL4_toref25_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL58_230927_101_full_wch0_aniso4_scale4AFS2PL4_toref31_wxy_nobg',
             r'D:\pkalugin\memmap_cache\PL58_230927_201_full_wch0_aniso4_scale4AFS2PL4_toref30_wxy_nobg',
             r'D:\pkalugin\memmap_cache\PL58_230927_301_full_wch0_aniso4_scale4AFS2PL4_toref30_wxy_nobg']
mov_means_l = [31,31]
pythonic_l = [#0,1,2,0,
              1,2]

doraw = True
n_cores = 30
nrg_thr = 0.95

# Parameters (unlikely to change)
n_range_lim = 10 # size of n_range below which SNR considered unreliable
Athresh = 0.05 # overlap threshold - automatically split anything below it
cr_thresh = 0.9 # component-raw correlation threshold below which component deemed suspicious quality
pb_thresh = 0.95 # component-best parent correlation threshold above which component deemed likely merge

In [8]:
# EDITED FURTHER 7/20/24 with pythonic order to properly coordinate matches!
# EDITED 5/5/24 with doraw option, also increased nrg_thr to 0.95 (not updated on Megatron/Beastmode)
data_dir_l = [#r'\\nasquatch\data\2p\peter\dipping\20230923_PL35\wells001\preprocessing\20230923_PL35_wells001_Tiara_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230923_PL35\wells001\preprocessing\20230923_PL35_wells001_Tiara_done_qc_doneqc']
             #r'\\nasquatch\data\2p\peter\dipping\20230923_PL35\wells001\preprocessing\20230923_PL35_wells001_Tiara_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230923_PL35\wells001\preprocessing\20230923_PL35_wells001_Tiara_done_qc_doneqc',
             r'\\nasquatch\data\2p\peter\dipping\20230923_PL35\wells001\preprocessing\20230923_PL35_wells001_Tiara_done_qc_doneqc',
             r'\\nasquatch\data\2p\peter\dipping\20230923_PL35\wells101\preprocessing\20230923_PL35_wells101_Yael_Done_qc_doneqc']
             #r'\\nasquatch\data\2p\peter\dipping\20230923_PL35\wells201\preprocessing\20230923_PL35_wells201_Yael_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230923_PL35\wells301\preprocessing\20230923_PL35_wells301_Yael_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230923_PL35\wells401\preprocessing\20230923_PL35_wells401_Yael_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230923_PL35\wells501\preprocessing\20230923_PL35_wells501_Yael_Done_qc_doneqc']
mmap_dir_l = [#r'D:\pkalugin\memmap_cache\PL35_230923_001_full_wch0_aniso4_scale4AFS2PL4_toref25_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL35_230923_001_full_wch0_aniso4_scale4AFS2PL4_toref25_wxy_nobg']
             #r'D:\pkalugin\memmap_cache\PL35_230923_001_full_wch0_aniso4_scale4AFS2PL4_toref25_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL35_230923_001_full_wch0_aniso4_scale4AFS2PL4_toref25_wxy_nobg',
             r'D:\pkalugin\memmap_cache\PL35_230923_001_full_wch0_aniso4_scale4AFS2PL4_toref25_wxy_nobg',
             r'D:\pkalugin\memmap_cache\PL35_230923_101_full_wch0_aniso4_scale4AFS2PL4_toref37_wxy_nobg']
             #r'D:\pkalugin\memmap_cache\PL35_230923_201_full_wch0_aniso4_scale4AFS2PL4_toref37_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL35_230923_301_full_wch0_aniso4_scale4AFS2PL4_toref26_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL35_230923_401_full_wch0_aniso4_scale4AFS2PL4_toref36_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL35_230923_501_full_wch0_aniso4_scale4AFS2PL4_toref31_wxy_nobg']
mov_means_l = [31,31]
pythonic_l = [#0,1,2,3,
              4,0]
              #1,2,3,4]

doraw = True
n_cores = 30
nrg_thr = 0.95

# Parameters (unlikely to change)
n_range_lim = 10 # size of n_range below which SNR considered unreliable
Athresh = 0.05 # overlap threshold - automatically split anything below it
cr_thresh = 0.9 # component-raw correlation threshold below which component deemed suspicious quality
pb_thresh = 0.95 # component-best parent correlation threshold above which component deemed likely merge

In [5]:
# EDITED FURTHER 7/20/24 with pythonic order to properly coordinate matches!
# EDITED 5/5/24 with doraw option, also increased nrg_thr to 0.95 (not updated on Megatron/Beastmode)
data_dir_l = [#r'\\nasquatch\data\2p\peter\dipping\20230916_PL54\wells001\preprocessing\20230916_PL54_wells001_jordan_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230916_PL54\wells001\preprocessing\20230916_PL54_wells001_jordan_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230916_PL54\wells001\preprocessing\20230916_PL54_wells001_jordan_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230916_PL54\wells001\preprocessing\20230916_PL54_wells001_jordan_done_qc_doneqc',
             r'\\nasquatch\data\2p\peter\dipping\20230916_PL54\wells001\preprocessing\20230916_PL54_wells001_jordan_done_qc_doneqc',
             r'\\nasquatch\data\2p\peter\dipping\20230916_PL54\wells101\preprocessing\20230916_PL54_wells101_jordan_done_qc_doneqc']
             #r'\\nasquatch\data\2p\peter\dipping\20230916_PL54\wells201\preprocessing\20230916_PL54_wells201_jordan_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230916_PL54\wells301\preprocessing\20230916_PL54_wells301_jordan_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230916_PL54\wells401\preprocessing\20230916_PL54_wells401_jordan_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230916_PL54\wells501\preprocessing\20230916_PL54_wells501_jordan_done_qc_doneqc']
mmap_dir_l = [#r'D:\pkalugin\memmap_cache\PL54_230916_001_full_wch0_aniso4_scale4AFS2PL4_toref18_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL54_230916_001_full_wch0_aniso4_scale4AFS2PL4_toref18_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL54_230916_001_full_wch0_aniso4_scale4AFS2PL4_toref18_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL54_230916_001_full_wch0_aniso4_scale4AFS2PL4_toref18_wxy_nobg',
             r'D:\pkalugin\memmap_cache\PL54_230916_001_full_wch0_aniso4_scale4AFS2PL4_toref18_wxy_nobg',
             r'D:\pkalugin\memmap_cache\PL54_230916_101_full_wch0_aniso4_scale4AFS2PL4_toref13_wxy_nobg']
             #r'D:\pkalugin\memmap_cache\PL54_230916_201_full_wch0_aniso4_scale4AFS2PL4_toref25_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL54_230916_301_full_wch0_aniso4_scale4AFS2PL4_toref19_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL54_230916_401_full_wch0_aniso4_scale4AFS2PL4_toref15_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL54_230916_501_full_wch0_aniso4_scale4AFS2PL4_toref35_wxy_nobg']
mov_means_l = [31,31]
pythonic_l = [#0,1,2,3,
              4,0]
              #1,2,3,4]

doraw = True
n_cores = 30
nrg_thr = 0.95

# Parameters (unlikely to change)
n_range_lim = 10 # size of n_range below which SNR considered unreliable
Athresh = 0.05 # overlap threshold - automatically split anything below it
cr_thresh = 0.9 # component-raw correlation threshold below which component deemed suspicious quality
pb_thresh = 0.95 # component-best parent correlation threshold above which component deemed likely merge

In [2]:
# EDITED FURTHER 7/20/24 with pythonic order to properly coordinate matches!
# EDITED 5/5/24 with doraw option, also increased nrg_thr to 0.95 (not updated on Megatron/Beastmode)
data_dir_l = [#r'\\nasquatch\data\2p\peter\dipping\20230925_PL58\wells001\preprocessing\20230925_PL58_wells001_jordan_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230925_PL58\wells001\preprocessing\20230925_PL58_wells001_jordan_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230925_PL58\wells001\preprocessing\20230925_PL58_wells001_jordan_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230925_PL58\wells001\preprocessing\20230925_PL58_wells001_jordan_done_qc_doneqc',
             r'\\nasquatch\data\2p\peter\dipping\20230925_PL58\wells101\preprocessing\20230925_PL58_wells101_Tiara_done_qc_doneqc',
             r'\\nasquatch\data\2p\peter\dipping\20230925_PL58\wells201\preprocessing\20230925_PL58_wells201_Tiara_done_qc_doneqc']
             #r'\\nasquatch\data\2p\peter\dipping\20230925_PL58\wells301\preprocessing\20230925_PL58_wells301_Tiara_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230925_PL58\wells401\preprocessing\20230925_PL58_wells401_Tiara_done_qc_doneqc']
mmap_dir_l = [#r'D:\pkalugin\memmap_cache\PL58_230925_001_full_wch0_aniso4_scale4AFS2PL4_toref30_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL58_230925_001_full_wch0_aniso4_scale4AFS2PL4_toref30_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL58_230925_001_full_wch0_aniso4_scale4AFS2PL4_toref30_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL58_230925_001_full_wch0_aniso4_scale4AFS2PL4_toref30_wxy_nobg',
             r'D:\pkalugin\memmap_cache\PL58_230925_101_full_wch0_aniso4_scale4AFS2PL4_toref15_wxy_nobg',
             r'D:\pkalugin\memmap_cache\PL58_230925_201_full_wch0_aniso4_scale4AFS2PL4_toref37_wxy_nobg']
             #r'D:\pkalugin\memmap_cache\PL58_230925_301_full_wch0_aniso4_scale4AFS2PL4_toref31_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL58_230925_401_full_wch0_aniso4_scale4AFS2PL4_toref31_wxy_nobg']
mov_means_l = [31,31]
pythonic_l = [#0,1,2,3,
              0,1]
              #2,3]

doraw = True
n_cores = 30
nrg_thr = 0.95

# Parameters (unlikely to change)
n_range_lim = 10 # size of n_range below which SNR considered unreliable
Athresh = 0.05 # overlap threshold - automatically split anything below it
cr_thresh = 0.9 # component-raw correlation threshold below which component deemed suspicious quality
pb_thresh = 0.95 # component-best parent correlation threshold above which component deemed likely merge

In [2]:
# EDITED FURTHER 7/20/24 with pythonic order to properly coordinate matches!
# EDITED 5/5/24 with doraw option, also increased nrg_thr to 0.95 (not updated on Megatron/Beastmode)
data_dir_l = [#r'\\nasquatch\data\2p\peter\slice\20230801_SL1PL35\slice\preprocessing\20230801_SL1PL35_slice_maddy_done_qc_done',
             #r'\\nasquatch\data\2p\peter\slice\20230801_SL1PL35\wells101\preprocessing\20230801_SL1PL35_wells101_maddy_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\slice\20230824_SL1PL60\slice\preprocessing\20230824_SL1PL60_slice_charlotte_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\slice\20230824_SL1PL60\slice\preprocessing\20230824_SL1PL60_slice_charlotte_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\slice\20230824_SL1PL60\wells001\preprocessing\20230824_SL1PL60_wells001_charlotte_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\slice\20230824_SL1PL60\wells101\preprocessing\20230824_SL1PL60_wells101_charlotte_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\slice\20230830_SL1PL28\slice\preprocessing\20230830_SL1PL28_slice_maddy_done_qc_done',
             #r'\\nasquatch\data\2p\peter\slice\20230830_SL1PL28\wells001\preprocessing\20230830_SL1PL28_wells001_Done_qc_doneqc',
             r'\\nasquatch\data\2p\peter\dipping\20230823_PL28\wells001\preprocessing\20230823_PL28_wells001_Tiara_done_qc_doneqc',
             r'\\nasquatch\data\2p\peter\dipping\20230823_PL28\wells001\preprocessing\20230823_PL28_wells001_Tiara_done_qc_doneqc',
             r'\\nasquatch\data\2p\peter\dipping\20230823_PL28\wells101\preprocessing\20230823_PL28_wells101_Tiara_done_qc_doneqc',
             r'\\nasquatch\data\2p\peter\dipping\20230823_PL28\wells201\preprocessing\20230823_PL28_wells201_Tiara_done_qc_doneqc',
             r'\\nasquatch\data\2p\peter\dipping\20230916_PL48\wells001\preprocessing\20230916_PL48_wells001_Tiara_done_qc_doneqc',
             r'\\nasquatch\data\2p\peter\dipping\20230916_PL48\wells101\preprocessing\20230916_PL48_wells101_Tiara_done_qc_doneqc']
             #r'\\nasquatch\data\2p\peter\dipping\20230920_PL58\wells001\preprocessing\20230920_PL58_wells001_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230920_PL58\wells001\preprocessing\20230920_PL58_wells001_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230920_PL58\wells001\preprocessing\20230920_PL58_wells001_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230920_PL58\wells101\preprocessing\20230920_PL58_wells101_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230920_PL58\wells201\preprocessing\20230920_PL58_wells201_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230920_PL58\wells301\preprocessing\20230920_PL58_wells301_Done_qc_doneqc']
mmap_dir_l = [#r'D:\pkalugin\memmap_cache\SL1PL35_230801_001_noartifact_wch0_aniso4_scale4.4AFS2PL4_toref27_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL35_230801_101_full_wch0_aniso4_scale4AFS2PL4_toref76_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\SL1PL60_230824_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref22_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\SL1PL60_230824_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref22_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL60_230824_001_full_wch1_aniso4_scale4AFS2PL4_toref18_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL60_230824_101_full_wch0_aniso4_scale4AFS2PL4_toref20_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\SL1PL28_230830_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref28_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL28_230830_001_full_wch0_aniso4_scale4AFS2PL4_toref52_wxy_nobg',
             r'D:\pkalugin\memmap_cache\PL28_230823_001_full_wch0_aniso4_scale4AFS2PL4_toref18_wxy_nobg',
             r'D:\pkalugin\memmap_cache\PL28_230823_001_full_wch0_aniso4_scale4AFS2PL4_toref18_wxy_nobg',
             r'D:\pkalugin\memmap_cache\PL28_230823_101_full_wch0_aniso4_scale4AFS2PL4_toref15_wxy_nobg',
             r'D:\pkalugin\memmap_cache\PL28_230823_201_full_wch0_aniso4_scale4AFS2PL4_toref46_wxy_nobg',
             r'D:\pkalugin\memmap_cache\PL48_230916_001_full_wch0_aniso4_scale4AFS2PL4_toref25_wxy_nobg',
             r'D:\pkalugin\memmap_cache\PL48_230916_101_full_wch0_aniso4_scale4AFS2PL4_toref40_wxy_nobg']
             #r'D:\pkalugin\memmap_cache\PL58_230920_001_full_wch0_aniso4_scale4AFS2PL4_toref25_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL58_230920_001_full_wch0_aniso4_scale4AFS2PL4_toref25_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL58_230920_001_full_wch0_aniso4_scale4AFS2PL4_toref25_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL58_230920_101_full_wch0_aniso4_scale4AFS2PL4_toref15_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL58_230920_201_full_wch0_aniso4_scale4AFS2PL4_toref20_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL58_230920_301_full_wch0_aniso4_scale4AFS2PL4_toref40_wxy_nobg']
mov_means_l = [#52,52,
              #52,52,52,52,
              #52,52,
              52,52,52,52,
              31,31]
              #31,31,31,31,31,31]
pythonic_l = [#0,0,
             #0,1,0,1,
             #0,0,
             0,1,0,1,
             0,0]
             #0,1,2,0,1,2]

doraw = True
n_cores = 30
nrg_thr = 0.95

# Parameters (unlikely to change)
n_range_lim = 10 # size of n_range below which SNR considered unreliable
Athresh = 0.05 # overlap threshold - automatically split anything below it
cr_thresh = 0.9 # component-raw correlation threshold below which component deemed suspicious quality
pb_thresh = 0.95 # component-best parent correlation threshold above which component deemed likely merge

In [12]:
# EDITED FURTHER 7/20/24 with pythonic order to properly coordinate matches!
# EDITED 5/5/24 with doraw option, also increased nrg_thr to 0.95 (not updated on Megatron/Beastmode)
data_dir_l = [#r'\\nasquatch\data\2p\peter\slice\20230509_DA1PL53\slice\preprocessing\20230509_DA1PL53_slice_Yael_Done_qc_done',
             #r'\\nasquatch\data\2p\peter\slice\20230509_DA1PL53\wells101\preprocessing\20230509_DA1PL53_wells101_Yael_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\slice\20230606_SL1PL35\slice\preprocessing\20230606_SL1PL35_slice_maddy_done_qc_done',
             #r'\\nasquatch\data\2p\peter\slice\20230606_SL1PL35\washes\preprocessing\20230606_SL1PL35_washes_maddy_done_qc_done',
             #r'\\nasquatch\data\2p\peter\slice\20230607_SL1PL47\slice\preprocessing\20230607_SL1PL47_slice_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\slice\20230607_SL1PL47\wells001\preprocessing\20230607_SL1PL47_wells001_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\slice\20230817_SL1PL55\slice\preprocessing\20230817_SL1PL55_slice_maddy_done_qc_done',
             #r'\\nasquatch\data\2p\peter\slice\20230817_SL1PL55\slice\preprocessing\20230817_SL1PL55_slice_maddy_done_qc_done',
             #r'\\nasquatch\data\2p\peter\slice\20230817_SL1PL55\wells001\preprocessing\20230817_SL1PL55_wells001_maddy_done_qc_done',
             #r'\\nasquatch\data\2p\peter\slice\20230817_SL1PL55\wells101\preprocessing\20230817_SL1PL55_wells101_maddy_done_qc_done',
             #r'\\nasquatch\data\2p\peter\slice\20230822_SL2PL52\slice\preprocessing\20230822_SL2PL52_slice_maddy_done_qc_done',
             #r'\\nasquatch\data\2p\peter\slice\20230822_SL2PL52\wells001\preprocessing\20230822_SL2PL52_wells001_maddy_done_qc_doneqc',
             r'\\nasquatch\data\2p\peter\slice\20230831_SL1PL55\slice\preprocessing\20230831_SL1PL55_slice_Yael_Done_qc_done',
             r'\\nasquatch\data\2p\peter\slice\20230831_SL1PL55\wells001\preprocessing\20230831_SL1PL55_wells001_Yael_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230823_PL28\wells001\preprocessing\20230823_PL28_wells001_Tiara_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230823_PL28\wells001\preprocessing\20230823_PL28_wells001_Tiara_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230823_PL28\wells101\preprocessing\20230823_PL28_wells101_Tiara_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230823_PL28\wells201\preprocessing\20230823_PL28_wells201_Tiara_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230915_PL52\wells001\preprocessing\20230915_PL52_wells001_Yael_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230915_PL52\wells001\preprocessing\20230915_PL52_wells001_Yael_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230915_PL52\wells001\preprocessing\20230915_PL52_wells001_Yael_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230915_PL52\wells001\preprocessing\20230915_PL52_wells001_Yael_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230915_PL52\wells101\preprocessing\20230915_PL52_wells101_Yael_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230915_PL52\wells201\preprocessing\20230915_PL52_wells201_Yael_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230915_PL52\wells301\preprocessing\20230915_PL52_wells301_Yael_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\dipping\20230915_PL52\wells401\preprocessing\20230915_PL52_wells401_Yael_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\in_vivo\220701_PK49PL18\mouse\preprocessing\220701_PK49PL18_mouse_jordan_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\in_vivo\220701_PK49PL18\wells101\preprocessing\220701_PK49PL18_wells101_jordan_done_qc_doneqc',
             r'\\nasquatch\data\2p\peter\in_vivo\221018_PS10PL24\mouse\preprocessing\221018_PS10PL24_mouse_Yael_Done_qc_doneqc',
             r'\\nasquatch\data\2p\peter\in_vivo\221018_PS10PL24\wells101\preprocessing\221018_PS10PL24_wells101_Yael_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\in_vivo\230519_DG41PL52\mouse\preprocessing\230519_DG41PL52_mouse_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\in_vivo\230519_DG41PL52\wells101\preprocessing\230519_DG41PL52_wells101_Done_qc_done',
             r'\\nasquatch\data\2p\peter\in_vivo\230525_DG42PL39\mouse\preprocessing\230525_DG42PL39_mouse_Tiara_done_qc_doneqc',
             r'\\nasquatch\data\2p\peter\in_vivo\230525_DG42PL39\wells101\preprocessing\230525_DG42PL39_wells101_Tiara_done_qc_doneqc',
             r'\\nasquatch\data\2p\peter\in_vivo\230526_PK84PL50\mouse\preprocessing\230526_PK84PL50_mouse_Yael_Done_qc_done',
             r'\\nasquatch\data\2p\peter\in_vivo\230526_PK84PL50\wells101\preprocessing\230526_PK84PL50_wells101_Yael_Done_qc_done',
             #r'\\nasquatch\data\2p\peter\in_vivo\230617_DG70PL54\mouse\preprocessing\230617_DG70PL54_mouse_Yael_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\in_vivo\230617_DG70PL54\wells101\preprocessing\230617_DG70PL54_wells101_Yael_done_qc_done',
             #r'\\nasquatch\data\2p\peter\in_vivo\230707_DG72PL32\mouse\preprocessing\230707_DG72PL32_mouse_charlotte_done_qc_done',
             #r'\\nasquatch\data\2p\peter\in_vivo\230707_DG72PL32\wells101\preprocessing\230707_DG72PL32_wells101_jordan_done_qc_done',
             r'\\nasquatch\data\2p\peter\in_vivo\230726_DG47PL54\mouse\preprocessing\230726_DG47PL54_mouse_jordan_done_qc_doneqc',
             r'\\nasquatch\data\2p\peter\in_vivo\230726_DG47PL54\wells101\preprocessing\230726_DG47PL54_wells101_jordan_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\in_vivo\230810_DG87PL55\mouse\preprocessing\230810_DG87PL55_mouse_maddy_done_qc_done',
             #r'\\nasquatch\data\2p\peter\in_vivo\230810_DG87PL55\wells101\preprocessing\230810_DG87PL55_wells101_maddy_done_qc_done',
             #r'\\nasquatch\data\2p\peter\in_vivo\230902_DG90PL61\mouse\preprocessing\230902_DG90PL61_mouse_maddy_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\in_vivo\230902_DG90PL61\wells101\preprocessing\230902_DG90PL61_wells101_jordan_done_qc_doneqc',
             r'\\nasquatch\data\2p\peter\in_vivo\230909_DG88PL58\mouse\preprocessing\230909_DG88PL58_mouse_Yael_done_qc_done',
             r'\\nasquatch\data\2p\peter\in_vivo\230909_DG88PL58\wells\preprocessing\230909_DG88PL58_wells_jordan_done_qc_done',
             #r'\\nasquatch\data\2p\peter\in_vivo\231024_DG54PL61\mouse\preprocessing\231024_DG54PL61_mouse_Done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\in_vivo\231024_DG54PL61\wells\preprocessing\231024_DG54PL61_wells_maddy_done_qc_done',
             #r'\\nasquatch\data\2p\peter\in_vivo\231025_DG56PL62\mouse\preprocessing\231025_DG56PL62_mouse_jordan_done_qc_doneqc',
             #r'\\nasquatch\data\2p\peter\in_vivo\231025_DG56PL62\wells\preprocessing\231025_DG56PL62_wells_jordan_done_qc_done',
             r'\\nasquatch\data\2p\peter\in_vivo\231105_DG39PL32\mouse\preprocessing\231105_DG39PL32_mouse_maddy_done_qc_done',
             r'\\nasquatch\data\2p\peter\in_vivo\231105_DG39PL32\wells\preprocessing\231105_DG39PL32_wells_maddy_done_qc_done']
mmap_dir_l = [#r'D:\pkalugin\memmap_cache\DA1PL53_230509_001_wch0_aniso4_scale4AFS2PL4_toref10_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL53_230509_101_full_wch0_aniso4_scale4.4AFS2PL4_toref8_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\SL1PL35_230606_001_wch0_aniso4_scale4.4AFS2PL4_toref18_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL35_230606_001_wch0_aniso4_scale4.4AFS2PL4_toref9_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\SL1PL47_230607_001_wch0_aniso4_scale4.4AFS2PL4_toref15_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL47_230607_001_full_wch0_aniso4_scale4.4AFS2PL4_toref27_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\SL1PL55_230817_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref27_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\SL1PL55_230817_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref27_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL55_230817_001_full_wch0_aniso4_scale4AFS2PL4_toref18_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL55_230817_101_full_wch0_aniso4_scale4AFS2PL4_toref39_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\SL2PL52_230822_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref28_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL52_230822_001_full_wch0_aniso4_scale4AFS2PL4_toref52_wxy_nobg',
             r'D:\pkalugin\memmap_cache\SL1PL55_230831_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref28_wxy_nobg',
             r'D:\pkalugin\memmap_cache\PL55_230831_001_full_wch0_aniso4_scale4AFS2PL4_toref52_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL52_230915_001_full_wch0_aniso4_scale4AFS2PL4_toref18_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL52_230915_001_full_wch0_aniso4_scale4AFS2PL4_toref18_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL52_230915_001_full_wch0_aniso4_scale4AFS2PL4_toref18_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL52_230915_001_full_wch0_aniso4_scale4AFS2PL4_toref18_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL52_230915_101_full_wch0_aniso4_scale4AFS2PL4_toref15_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL52_230915_201_full_wch0_aniso4_scale4AFS2PL4_toref31_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL52_230915_301_full_wch0_aniso4_scale4AFS2PL4_toref25_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL52_230915_401_full_wch0_aniso4_scale4AFS2PL4_toref19_wxy_nobg',
             r'D:\pkalugin\memmap_cache\PS10PL24_221018_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref31_wxy_nobg',
             r'D:\pkalugin\memmap_cache\PS10PL24_221018_101_full_wch0_aniso4_scale4AFS2PL4_toref20_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\DG41PL52_230519_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref71_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\PL52_230519_101_full_wch0_aniso4_scale4AFS2PL4_toref1_wxy_nobg',
             r'D:\pkalugin\memmap_cache\DG42PL39_230525_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref70_wxy_nobg',
             r'D:\pkalugin\memmap_cache\PL39_230525_101_full_wch0_aniso4_scale4AFS2PL4_toref1_wxy_nobg',
             r'D:\pkalugin\memmap_cache\PK84PL50_230526_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref62_wxy_nobg',
             r'D:\pkalugin\memmap_cache\PL50_230526_101_full_wch0_aniso4_scale4AFS2PL4_toref1_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\DG70PL54_230617_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref71_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\DG70PL54_230617_101_full_wch0_aniso4_scale4AFS2PL4_toref40_wxy_nobg',
             r'D:\pkalugin\memmap_cache\DG47PL54_230726_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref71_wxy_nobg',
             r'D:\pkalugin\memmap_cache\DG47PL54_230726_101_full_wch0_aniso4_scale4AFS2PL4_toref30_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\DG87PL55_230810_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref94_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\DG87PL55_230810_101_full_wch0_aniso4_scale4AFS2PL4_toref48_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\DG90PL61_230902_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref75_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\DG90PL61_230902_101_full_wch0_aniso4_scale4AFS2PL4_toref38_wxy_nobg',
             r'D:\pkalugin\memmap_cache\DG88PL58_230909_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref71_wxy_nobg',
             r'D:\pkalugin\memmap_cache\DG88PL58_230909_101_full_wch1_aniso4_scale4AFS2PL4_toref34_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\DG54PL61_231024_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref71_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\DG54PL61_231024_101_full_wch1_aniso4_scale4AFS2PL4_toref34_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\DG56PL62_231025_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref85_wxy_nobg',
             #r'D:\pkalugin\memmap_cache\DG56PL62_231025_101_full_wch1_aniso4_scale4AFS2PL4_toref34_wxy_nobg',
             r'D:\pkalugin\memmap_cache\DG39PL32_231105_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref85_wxy_nobg',
             r'D:\pkalugin\memmap_cache\DG39PL32_231105_201_full_wch1_aniso4_scale4AFS2PL4_toref34_wxy_nobg']
mov_means_l = [#52,52,
              #52,52,
              #52,52,
              #52,52,52,52,
              #52,52,
              52,52,
              #31,31,31,31,
              #31,31,31,31,31,31,31,31,
              #52,52,
              52,52,
              #52,52,
              52,52,
              52,52,
              #52,52,
              #52,52,
              52,52,
              #52,52,
              #52,52,
              31,31,
              #52,52,
              #52,52,
              52,52]
pythonic_l = [#0,0,
             #0,0,
             #0,0,
             #0,1,0,1,
             #0,0,
             0,0,
             #0,1,0,1,
             #0,1,2,3,0,1,2,3,
             #0,0,
             0,0,
             #0,0,
             0,0,
             0,0,
             #0,0,
             #0,0,
             0,0,
             #0,0,
             #0,0,
             0,0,
             #0,0,
             #0,0,
             0,0]

doraw = True
n_cores = 30
nrg_thr = 0.95

# Parameters (unlikely to change)
n_range_lim = 10 # size of n_range below which SNR considered unreliable
Athresh = 0.05 # overlap threshold - automatically split anything below it
cr_thresh = 0.9 # component-raw correlation threshold below which component deemed suspicious quality
pb_thresh = 0.95 # component-best parent correlation threshold above which component deemed likely merge

In [2]:
# EDITED 5/5/24 with doraw option, also increased nrg_thr to 0.95 (not updated on Megatron/Beastmode)
data_dir_l = [r'\\nasquatch\data\2p\peter\dipping\20230922_PL28\wells001\preprocessing\20230922_PL28_wells001_jordan_done_qc_done']
mmap_dir_l = [r'D:\pkalugin\memmap_cache\PL28_230922_001_full_wch1_aniso4_scale4AFS2PL4_toref35_wxy_nobg']
mov_means_l = [31]

doraw = True
n_cores = 30
nrg_thr = 0.95

# Parameters (unlikely to change)
n_range_lim = 10 # size of n_range below which SNR considered unreliable
Athresh = 0.05 # overlap threshold - automatically split anything below it
cr_thresh = 0.9 # component-raw correlation threshold below which component deemed suspicious quality
pb_thresh = 0.95 # component-best parent correlation threshold above which component deemed likely merge

In [17]:
# EDITED 5/5/24 with doraw option, also increased nrg_thr to 0.95 (not updated on Megatron/Beastmode)
data_dir_l = [r'\\nasquatch\data\2p\peter\dipping\20230922_PL28\wells401\preprocessing\20230922_PL28_wells401_Tiara_done_qc_done',
             r'\\nasquatch\data\2p\peter\dipping\20230922_PL28\wells001\preprocessing\20230922_PL28_wells001_jordan_done_qc_done',
             r'\\nasquatch\data\2p\peter\dipping\20230922_PL28\wells201\preprocessing\20230922_PL28_wells201_maddy_done_qc_done',
             r'\\nasquatch\data\2p\peter\slice\20230829_SL1PL56\slice\preprocessing\20230829_SL1PL56_slice_Tiara_done_qc_done',
             r'\\nasquatch\data\2p\peter\slice\20230829_SL1PL56\wells001\preprocessing\20230829_SL1PL56_wells001_done_qc_done',
             r'\\nasquatch\data\2p\peter\in_vivo\231106_DG52PL68\mouse\preprocessing\231106_DG52PL68_mouse_jordan_done_qc_peter_done',
             r'\\nasquatch\data\2p\peter\in_vivo\231106_DG52PL68\wells\preprocessing\231106_DG52PL68_wells_jordan_done_qc_peter_done']
mmap_dir_l = [r'D:\pkalugin\memmap_cache\PL28_230922_401_full_wch1_aniso4_scale4AFS2PL4_toref18_wxy_nobg',
             r'D:\pkalugin\memmap_cache\PL28_230922_001_full_wch1_aniso4_scale4AFS2PL4_toref35_wxy_nobg',
             r'D:\pkalugin\memmap_cache\PL28_230922_201_full_wch1_aniso4_scale4AFS2PL4_toref19_wxy_nobg',
             r'D:\pkalugin\memmap_cache\SL1PL56_230829_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref28_wxy_nobg',
             r'D:\pkalugin\memmap_cache\PL56_230829_001_full_wch1_aniso4_scale4AFS2PL4_toref52_wxy_nobg',
             r'D:\pkalugin\memmap_cache\DG52PL68_231106_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref85_wxy_nobg',
             r'D:\pkalugin\memmap_cache\DG52PL68_231106_101_full_wch1_aniso4_scale4AFS2PL4_toref34_wxy_nobg']
mov_means_l = [31,
              31,
              31,
              52,
              52,
              52,
              52]

doraw = True
n_cores = 30
nrg_thr = 0.95

# Parameters (unlikely to change)
n_range_lim = 10 # size of n_range below which SNR considered unreliable
Athresh = 0.05 # overlap threshold - automatically split anything below it
cr_thresh = 0.9 # component-raw correlation threshold below which component deemed suspicious quality
pb_thresh = 0.95 # component-best parent correlation threshold above which component deemed likely merge

In [13]:
# EDITED 5/7/24 to take post-mask matched dictionaries
data_dir_l = [r'\\nasquatch\data\2p\peter\slice\20230829_SL1PL56\slice\preprocessing\20230829_SL1PL56_slice_Tiara_done_qc_done',
             r'\\nasquatch\data\2p\peter\slice\20230829_SL1PL56\wells001\preprocessing\20230829_SL1PL56_wells001_done_qc_done',
             r'\\nasquatch\data\2p\peter\dipping\20230922_PL28\wells001\preprocessing\20230922_PL28_wells001_jordan_done_qc_done',
             r'\\nasquatch\data\2p\peter\dipping\20230922_PL28\wells201\preprocessing\20230922_PL28_wells201_maddy_done_qc_done']
mmap_dir_l = [r'D:\pkalugin\memmap_cache\SL1PL56_230829_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref28_wxy_nobg',
             r'D:\pkalugin\memmap_cache\PL56_230829_001_full_wch1_aniso4_scale4AFS2PL4_toref52_wxy_nobg',
             r'D:\pkalugin\memmap_cache\PL28_230922_001_full_wch1_aniso4_scale4AFS2PL4_toref35_wxy_nobg',
             r'D:\pkalugin\memmap_cache\PL28_230922_201_full_wch1_aniso4_scale4AFS2PL4_toref19_wxy_nobg']
mov_means_l = [52,
              52,
              31,
              31]

doraw = False #PAY ATTENTION HERE
n_cores = 30
nrg_thr = 0.95

# Parameters (unlikely to change)
n_range_lim = 10 # size of n_range below which SNR considered unreliable
Athresh = 0.05 # overlap threshold - automatically split anything below it
cr_thresh = 0.9 # component-raw correlation threshold below which component deemed suspicious quality
pb_thresh = 0.95 # component-best parent correlation threshold above which component deemed likely merge

In [5]:
# EDITED 5/7/24 to take post-mask matched dictionaries
data_dir_l = [r'\\nasquatch\data\2p\peter\in_vivo\231106_DG52PL68\mouse\preprocessing\231106_DG52PL68_mouse_jordan_done_qc_peter_done',
             r'\\nasquatch\data\2p\peter\in_vivo\231106_DG52PL68\wells\preprocessing\231106_DG52PL68_wells_jordan_done_qc_peter_done']
mmap_dir_l = [r'D:\pkalugin\memmap_cache\DG52PL68_231106_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref85_wxy_nobg',
             r'D:\pkalugin\memmap_cache\DG52PL68_231106_101_full_wch1_aniso4_scale4AFS2PL4_toref34_wxy_nobg']
mov_means_l = [52,
              52]

doraw = False
n_cores = 30
nrg_thr = 0.95

# Parameters (unlikely to change)
n_range_lim = 10 # size of n_range below which SNR considered unreliable
Athresh = 0.05 # overlap threshold - automatically split anything below it
cr_thresh = 0.9 # component-raw correlation threshold below which component deemed suspicious quality
pb_thresh = 0.95 # component-best parent correlation threshold above which component deemed likely merge

In [2]:
# EDITED 5/7/24 to take post-mask matched dictionaries
data_dir_l = [r'\\nasquatch\data\2p\peter\in_vivo\231105_DG39PL32\mouse\preprocessing\231105_DG39PL32_mouse_maddy_done',
             r'\\nasquatch\data\2p\peter\in_vivo\231105_DG39PL32\wells\preprocessing\231105_DG39PL32_wells_maddy_done']
mmap_dir_l = [r'D:\pkalugin\memmap_cache\DG39PL32_231105_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref85_wxy_nobg',
             r'D:\pkalugin\memmap_cache\DG39PL32_231105_201_full_wch1_aniso4_scale4AFS2PL4_toref34_wxy_nobg']
mov_means_l = [52,
              52]

doraw = False
n_cores = 30
nrg_thr = 0.95

# Parameters (unlikely to change)
n_range_lim = 10 # size of n_range below which SNR considered unreliable
Athresh = 0.05 # overlap threshold - automatically split anything below it
cr_thresh = 0.9 # component-raw correlation threshold below which component deemed suspicious quality
pb_thresh = 0.95 # component-best parent correlation threshold above which component deemed likely merge

In [24]:
for m in trange(0,len(data_dir_l)):
    data_dir = data_dir_l[m]
    mmap_dir = mmap_dir_l[m]
    mov_means = mov_means_l[m]
    pythonic_n = pythonic_l[m]
    
    data_dir = data_dir.replace(f'\\',f'/')
    mmap_dir = mmap_dir.replace(f'\\',f'/')
    
    ## Loading all the inputs
    os.chdir(data_dir)
    #cnmf_path = caiman_datadir()+'/example_movies/demoMovie3DYxxbnobg_20240318170305_cnmf.hdf5'
    cnmf_path = os.path.join(data_dir, 'ch0_means_movie_nobg_cnmf.hdf5')
    
    # CNMFE model
    cnmf_model = cnmf.load_CNMF(cnmf_path, 
                                n_processes=1,
                                dview=None)
    #print(f"Successfully loaded CNMF model")
    
    mc_memmapped_fname = [i for i in os.listdir() if 'memmap__' in i][0]
    Yr, dims, T = cm.load_memmap(mc_memmapped_fname)
    #images = np.array(np.reshape(Yr.T, [T] + list(dims), order='F')) 
    images = np.reshape(Yr.T, [T] + list(dims), order='F')
    #print(f"Successfully loaded data")
    
    #d = cnmf_model.estimates.A.shape[0]
    #dims = cnmf_model.estimates.dims
    #axis = 2
    #order = list(range(4))
    #order.insert(0, order.pop(axis))
    #index_permut = np.reshape(np.arange(d), dims, order='F').transpose(
    #        order[:-1]).reshape(d, order='F')
    #A = csc_matrix(cnmf_model.estimates.A)[index_permut, :]
    #dims = tuple(np.array(dims)[order[:3]])
    #d1, d2, d3 = dims
    #nr, T = cnmf_model.estimates.C.shape
    #image_cells = np.array(A.mean(axis=1)).reshape(dims, order='F')
    #coors = get_contours(A, dims, thr=Cthr)
    coors = load_pickle(os.path.join(data_dir, 'ch0_means_movie_nobg_coors.pickle'))
    
    cc = [[l for l in n['coordinates']] for n in coors] # x,y values of contour coordinates for each component
    cc1 = [[(l[:, 0]) for l in n['coordinates']] for n in coors] # x values of contour coordinates for each component
    cc2 = [[(l[:, 1]) for l in n['coordinates']] for n in coors] # y values of contour coordinates for each component
    length = np.ravel([list(map(len, cc)) for cc in cc1])
    shapes = [[np.vstack([np.append(i,np.flip(pt)) for pt in cc[j][i]]) for i in range(len(cc[j]))] for j in range(len(cc))]
    
    # Line up all static inputs
    SNRs = cnmf_model.estimates.SNR_comp
    SNR_min = cnmf_model.estimates.SNRmin
    SOL = np.argsort(-SNRs)
    spcomps = np.reshape(cnmf_model.estimates.A.toarray(),cnmf_model.estimates.dims + (-1,),order='F')
    spcomps = spcomps.transpose([3,2,0,1])
    images2 = images.transpose([0,3,1,2])
    #SOL = np.argsort(-cnmf_model.estimates.SNR_comp) 
    C = cnmf_model.estimates.C
    CY = cnmf_model.estimates.C + cnmf_model.estimates.YrA # temporal loadings
    R = cnmf_model.estimates.Craw # masks applied to raw movie
    
    CYsav = cnmf_model.estimates.CYsav # smoothened CY curve
    def sav_calc(sraw):
        return sg.savgol_filter(sraw,3,1)
    Rsav = np.zeros(R.shape)
    for i in range(R.shape[0]):
        Rsav[i,:] = sav_calc(R[i,:])
    CYsavsort = np.sort(CYsav,axis=1)
    CYsavb10 = np.mean(CYsavsort[:,:int(np.ceil(CYsavsort.shape[1]/10))],axis=1) # bottom 10% mean
    Rsavsort = np.sort(Rsav,axis=1)
    Rsavb10 = np.mean(Rsavsort[:,:int(np.ceil(Rsavsort.shape[1]/10))],axis=1) # bottom 10% mean
    n_range = cnmf_model.estimates.n_range
    if n_range is None:
        CYf = CYsavb10
        Rf = Rsavb10
    else:
        CYf = np.mean(CY[:,n_range],axis=1)
        Rf = np.mean(R[:,n_range],axis=1)
    
    Cn = cnmf_model.estimates.Cn # correlation image (not necessary)
    keepargs = cnmf_model.estimates.keepargs
    SOL = np.array([x for x in list(SOL) if x in list(keepargs)]) # initial search order list
    
    A1 = csc_matrix(cnmf_model.estimates.A)
    nr = A1.shape[1]
    A_corr = scipy.sparse.triu(A1.T * A1)
    A_corr.setdiag(0)
    A_corr = A_corr.tocsc()
    C_corr = scipy.sparse.lil_matrix(A_corr.shape)
    for ii in range(nr):
        overlap_indices = scipy.sparse.find(A_corr[ii, :])[1][scipy.sparse.find(A_corr[ii, :])[2]>Athresh]
        if len(overlap_indices) > 0:
                # we chesk the correlation of the calcium traces for each overlapping components
            corr_values = [scipy.stats.pearsonr(C[ii, :], C[jj, :])[
                0] for jj in overlap_indices]
            C_corr[ii, overlap_indices] = corr_values
    C_tot = C_corr + C_corr.T
    CYR_corr = np.zeros(nr)
    for ii in range(nr):
        CYR_corr[ii] = scipy.stats.pearsonr(R[ii,:],CY[ii,:])[0]
    
    # Initialize all running variables in a single dictionary - just lists of arguments/component IDs (SOL, CSL, CMG, CKG, saved merge groups, trash)
    # check if saved file exists - load that if it does, else instantiate new vars_dict!
    if pythonic_n == 0:
        save_path = f'ch0_means_movie_nobg_compfilt_matched.pickle'
    else:
        save_path = f'ch0_means_movie_nobg_compfilt_matched{pythonic_n:01}.pickle'
    vars_dict1 = load_pickle(save_path)
    
    #%% start a cluster for parallel processing (if a cluster already exists it will be closed and a new session will be opened)
    n_processes = n_cores
    if 'dview' in locals():
        cm.stop_server(dview=dview)
    c, dview, n_processes = cm.cluster.setup_cluster(
        backend='multiprocessing', n_processes=n_processes, single_thread=False)
    
    parameter_dict = {#'fnames': cnmf_model.params.data['fnames'][0].decode('UTF-8'),
                      #'dims': cnmf_model.params.data['dims'],
                      'p': cnmf_model.params.temporal['p'],
                      'nb': cnmf_model.params.temporal['nb'],
                      'rf': cnmf_model.params.patch['rf'],
                      'K': cnmf_model.params.init['K'], 
                      'gSig': cnmf_model.params.init['gSig'],
                      'gSiz': cnmf_model.params.init['gSiz'],
                      'stride': cnmf_model.params.patch['stride'],
                      'method_init': cnmf_model.params.init['method_init'],
                      'rolling_sum': cnmf_model.params.init['rolling_sum'], #important
                      'rolling_length': cnmf_model.params.init['rolling_length'],
                      'only_init': cnmf_model.params.patch['only_init'], #also keep an eye on this
                      'ssub': cnmf_model.params.init['ssub'],
                      'tsub': cnmf_model.params.init['tsub'],
                      'merge_thr': cnmf_model.params.merging['merge_thr'], 
                      'bas_nonneg': cnmf_model.params.temporal['bas_nonneg'],
                      'kernel': cnmf_model.params.init['kernel'],
                      'nIter': cnmf_model.params.init['nIter']
                     }
    
    parameters = params.CNMFParams(params_dict=parameter_dict) # CNMFParams is the parameters class
    
    cnmf_model_merged = cnmf.CNMF(n_processes, 
                           params=parameters, 
                           dview=dview)
    cnmf_model_merged.params.data['dims'] = cnmf_model.params.data['dims']
    cnmf_model_merged.params.spatial['n_pixels_per_process'] = cnmf_model.params.spatial['n_pixels_per_process']
    cnmf_model_merged.params.preprocess['n_pixels_per_process'] = cnmf_model.params.preprocess['n_pixels_per_process']
    cnmf_model_merged.params.spatial['nrgthr'] = nrg_thr
    cnmf_model_merged.mmap_file = cnmf_model.mmap_file
    cnmf_model_merged.estimates = deepcopy(cnmf_model.estimates)
    
    cnmf_model_merged.estimates.idx_components = flatten_extend(vars_dict1["SMG"])
    cnmf_model_merged.estimates.idx_components_bad = np.setdiff1d(range(cnmf_model.estimates.C.shape[0]),flatten_extend(vars_dict1["SMG"]))
    cnmf_model_merged.estimates.manual_merge(vars_dict1["SMG"],cnmf_model.params)
    cnmf_model_merged.estimates.select_components(use_object=True)

    print(cnmf_model.estimates.C.shape)
    print(cnmf_model_merged.estimates.C.shape)
    
    Yr = np.transpose(np.reshape(images, (T, -1), order='F'))
    Yr.filename = images.filename
    Yr = cnmf_model_merged.preprocess(Yr)
    cnmf_model_merged.update_temporal(Yr, use_init=False)
    cnmf_model_merged.estimates.normalize_components()
    print(cnmf_model_merged.estimates.C.shape)

    if pythonic_n == 0:
        save_path_merged = os.path.join(data_dir, f'ch0_means_movie_nobg_cnmf_merged_matched.hdf5')
    else:
        save_path_merged = os.path.join(data_dir, f'ch0_means_movie_nobg_cnmf_merged_matched{pythonic_n:01}.hdf5')
    cnmf_model_merged.save(save_path_merged)
    print("Saved merged model")
    
    #~2 hrs on colab05, XX on beastmode, 3 hrs on megatron
    cnmf_model_merged_refit = cnmf_model_merged.refit(images)
    print(cnmf_model_merged_refit.estimates.C.shape)
    
    # save merged and refit outputs
    #print("Saving CNMF model...")

    if pythonic_n == 0:
        save_path_merged_refit = os.path.join(data_dir, f'ch0_means_movie_nobg_cnmf_merged_refit_matched.hdf5')
    else:
        save_path_merged_refit = os.path.join(data_dir, f'ch0_means_movie_nobg_cnmf_merged_refit_matched{pythonic_n:01}.hdf5')
    cnmf_model_merged_refit.save(save_path_merged_refit)
    print("Saved refit model")

    if doraw:
        # apply merged/refit model to raw movie
        fname_mmap = mmap_dir + f'/' + os.listdir(mmap_dir)[0]
        Yr2, dims2, T2 = cm.load_memmap(fname_mmap)
        images2 = np.reshape(Yr2.T, [T2] + list(dims2), order='F')
        
        K = cnmf_model_merged_refit.estimates.C.shape[0]
        T = cnmf_model_merged_refit.estimates.C.shape[1]*mov_means
        Cin = cm.source_extraction.cnmf.initialization.resize(cnmf_model_merged_refit.estimates.C, [K,T])
        if Cin.shape[1] < images2.shape[0]:
            Cin = np.concatenate((Cin,np.repeat(Cin[...,Cin.shape[1]-1][:,np.newaxis],images2.shape[0]-Cin.shape[1],axis=1)),axis=1,dtype=Cin.dtype)    
        YrAin = cm.source_extraction.cnmf.initialization.resize(cnmf_model_merged_refit.estimates.YrA, [K,T])
        if YrAin.shape[1] < images2.shape[0]:
            YrAin = np.concatenate((YrAin,np.repeat(YrAin[...,YrAin.shape[1]-1][:,np.newaxis],images2.shape[0]-YrAin.shape[1],axis=1)),axis=1,dtype=YrAin.dtype)  
        
        cnmf_model_mr_raw = cnmf.CNMF(n_processes, 
                               params=parameters, 
                               dview=dview)
        cnmf_model_mr_raw.estimates = deepcopy(cnmf_model_merged_refit.estimates)
        cnmf_model_mr_raw.estimates.C = Cin
        cnmf_model_mr_raw.estimates.YrA = YrAin
        cnmf_model_mr_raw.estimates.b = None
        cnmf_model_mr_raw.estimates.f = None
        cnmf_model_mr_raw.mmap_file = cnmf_model.mmap_file
        
        #~2.5 hrs on colab05, XX on beastmode, 5.5 hrs on megatron
        cnmf_model_mr_raw_refit = cnmf_model_mr_raw.refit(images2)
        print(cnmf_model_mr_raw_refit.estimates.C.shape)
        
        # save merged and refit outputs
        #print("Saving CNMF model...")
        if pythonic_n == 0:
            save_path_mr_raw = os.path.join(data_dir, f'ch0_means_movie_nobg_cnmf_mr_raw_matched.hdf5')
            save_path_mr_raw_refit = os.path.join(data_dir, f'ch0_means_movie_nobg_cnmf_mr_raw_refit_matched.hdf5')
        else:
            save_path_mr_raw = os.path.join(data_dir, f'ch0_means_movie_nobg_cnmf_mr_raw_matched{pythonic_n:01}.hdf5')
            save_path_mr_raw_refit = os.path.join(data_dir, f'ch0_means_movie_nobg_cnmf_mr_raw_refit_matched{pythonic_n:01}.hdf5')
        cnmf_model_mr_raw.save(save_path_mr_raw)
        cnmf_model_mr_raw_refit.save(save_path_mr_raw_refit)
        
        print("Saved raw/refit models")

# STOP CLUSTER
cm.stop_server(dview=dview)

  0%|          | 0/2 [00:00<?, ?it/s]

D:\pkalugin\miniconda3\envs\caiman_full_102\lib\site-packages\scipy\signal\_spectral_py.py:600: UserWarning: nperseg = 256 is greater than input length  = 142, using nperseg = 142
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,


*** Variable lam has not the same number of components as A ***
*** Variable SNR_comp has not the same number of components as A ***
(893, 142)
(69, 142)
(69, 142)
Saved merged model
(5048500, 69, 9, <1x1 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>)
(5048500, 69, 10, 58.6536232167946)
(5048500, 69, 0, 887326406.4391893)
(5048500, 69, 11, 19111675.521667253)
(5048500, 69, 1, 14227354.277263965)
(5048500, 69, 2, 14227354.277263965)


D:\pkalugin\miniconda3\envs\caiman_full_102\lib\site-packages\scipy\signal\_spectral_py.py:600: UserWarning: nperseg = 256 is greater than input length  = 142, using nperseg = 142
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,


(5048500, 69, 9, <1x1 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>)
(5048500, 69, 10, 14227354.277263965)
(5048500, 69, 0, 1029774337.9234641)
(5048500, 69, 11, 19497794.717741452)
(5048500, 69, 1, 14911617.235695641)
(5048500, 69, 2, 14911617.235695641)


...

(4239375, 69, 9, <1x1 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>)
(4239375, 69, 10, 5695700.295889036)
(4239375, 69, 0, 667939400.6936408)
(4239375, 69, 11, 8331867.17938929)
(4239375, 69, 1, 6336859.846610339)
(4239375, 69, 2, 6336859.846610339)


D:\pkalugin\miniconda3\envs\caiman_full_102\lib\site-packages\scipy\signal\_spectral_py.py:600: UserWarning: nperseg = 256 is greater than input length  = 114, using nperseg = 114
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,
D:\pkalugin\miniconda3\envs\caiman_full_102\lib\site-packages\scipy\sparse\_dia.py:277: RuntimeWarning: divide by zero encountered in remainder
  c = np.arange(num_rows, dtype=np.intc) - (offsets % max_dim)[:, None]


(69, 114)
Saved refit model
(4239375, 69, 9, <1x1 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>)
(4239375, 69, 10, 58.73349687431124)
(4239375, 69, 0, 5404711712.448495)
(4239375, 69, 11, 9227334.290025136)
(4239375, 69, 1, 7048576.5495838495)
(4239375, 69, 2, 7048576.5495838495)
(4239375, 69, 9, <1x1 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>)
(4239375, 69, 10, 7048576.5495838495)
(4239375, 69, 0, 5945231116.890554)
(4239375, 69, 11, 10279098.195735773)
(4239375, 69, 1, 7899251.556867968)
(4239375, 69, 2, 7899251.556867968)


D:\pkalugin\miniconda3\envs\caiman_full_102\lib\site-packages\scipy\sparse\_dia.py:277: RuntimeWarning: divide by zero encountered in remainder
  c = np.arange(num_rows, dtype=np.intc) - (offsets % max_dim)[:, None]


(69, 5928)
Saved raw/refit models


In [4]:
vars_dict1

{'SOL': [],
 'CSL': [],
 'CMG': [],
 'CKG': [],
 'SMG': [[547, 367, 411],
  [280, 437],
  [429, 406, 395],
  [383, 193],
  [825],
  [483],
  [413, 441],
  [458, 276],
  [552, 374, 386],
  [26],
  [392, 576, 409],
  [239],
  [485, 451, 480, 490, 459, 494, 497],
  [548],
  [561, 555, 551, 562],
  [241, 251, 260],
  [763],
  [293],
  [577],
  [655],
  [773, 768, 797],
  [805],
  [94, 291, 78],
  [430, 257, 425, 443],
  [685, 674, 684],
  [324],
  [223],
  [54, 85, 59, 44],
  [219],
  [401],
  [394, 388, 230, 402, 414],
  [666],
  [314, 302, 342, 331, 338, 301, 315],
  [802, 795],
  [434, 442],
  [649, 461],
  [478, 665, 453, 675, 644],
  [188, 361, 391],
  [217, 397, 227],
  [426, 420, 445, 431, 435],
  [373, 385],
  [107, 125, 116, 127, 105, 133, 323, 129, 138],
  [284],
  [609, 574],
  [547, 367, 411],
  [280, 437],
  [429, 406, 395],
  [383, 193],
  [825],
  [483],
  [413, 441],
  [458, 276],
  [552, 374, 386],
  [26],
  [392, 576, 409],
  [239],
  [485, 451, 480, 490, 459, 494, 497],


In [5]:
ln = np.sum(np.array([len(comp) for comp in vars_dict1['SMG']]))
ids = set.union(*[set(comp) for comp in vars_dict1['SMG']])

In [6]:
ln

684

In [7]:
len(ids)

114